RAG Pipelines- Data Ingestion to Vector DB Pipeline


In [40]:

import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [41]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDF in data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: test1.pdf
Loaded 26 pages

Total documents loaded: 26


In [42]:
all_pdf_documents


[Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creationdate': '2022-09-29T17:54:00+02:00', 'author': 'ISO', 'license': 'Information Handling Services, 2022', 'moddate': '2022-10-26T19:22:40+08:00', 'title': 'ISO/IEC 27001:2022', 'trapped': '/False', 'source': '..\\data\\pdf\\test1.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1', 'source_file': 'test1.pdf', 'file_type': 'pdf'}, page_content="Information security, cybersecurity \nand privacy protection — Information \nsecurity management systems — \nRequirements\nSécurité de l'information, cybersécurité et protection de la vie \nprivée — Systèmes de management de la sécurité de l'information — \nExigences\nINTERNATIONAL \nSTANDARD\nISO/IEC \n27001\nThird edition  \n2022-10\nReference number \nISO/IEC 27001:2022(E)\n© ISO/IEC 2022\n--``,,,,,``````,,,,,`,`,`,`,,`,-`-`,,`,,`,`,,`---"),
 Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creati

In [43]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [44]:

chunks=split_documents(all_pdf_documents)
chunks

Split 26 documents into 76 chunks

Example chunk:
Content: Information security, cybersecurity 
and privacy protection — Information 
security management systems — 
Requirements
Sécurité de l'information, cybersécurité et protection de la vie 
privée — Systèm...
Metadata: {'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creationdate': '2022-09-29T17:54:00+02:00', 'author': 'ISO', 'license': 'Information Handling Services, 2022', 'moddate': '2022-10-26T19:22:40+08:00', 'title': 'ISO/IEC 27001:2022', 'trapped': '/False', 'source': '..\\data\\pdf\\test1.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1', 'source_file': 'test1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creationdate': '2022-09-29T17:54:00+02:00', 'author': 'ISO', 'license': 'Information Handling Services, 2022', 'moddate': '2022-10-26T19:22:40+08:00', 'title': 'ISO/IEC 27001:2022', 'trapped': '/False', 'source': '..\\data\\pdf\\test1.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1', 'source_file': 'test1.pdf', 'file_type': 'pdf'}, page_content="Information security, cybersecurity \nand privacy protection — Information \nsecurity management systems — \nRequirements\nSécurité de l'information, cybersécurité et protection de la vie \nprivée — Systèmes de management de la sécurité de l'information — \nExigences\nINTERNATIONAL \nSTANDARD\nISO/IEC \n27001\nThird edition  \n2022-10\nReference number \nISO/IEC 27001:2022(E)\n© ISO/IEC 2022\n--``,,,,,``````,,,,,`,`,`,`,,`,-`-`,,`,,`,`,,`---"),
 Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creati

embedding And vectorStoreDB


In [45]:

import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Model loaded successfully. Embedding dimension: 384


In [47]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [48]:
chunks

[Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creationdate': '2022-09-29T17:54:00+02:00', 'author': 'ISO', 'license': 'Information Handling Services, 2022', 'moddate': '2022-10-26T19:22:40+08:00', 'title': 'ISO/IEC 27001:2022', 'trapped': '/False', 'source': '..\\data\\pdf\\test1.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1', 'source_file': 'test1.pdf', 'file_type': 'pdf'}, page_content="Information security, cybersecurity \nand privacy protection — Information \nsecurity management systems — \nRequirements\nSécurité de l'information, cybersécurité et protection de la vie \nprivée — Systèmes de management de la sécurité de l'information — \nExigences\nINTERNATIONAL \nSTANDARD\nISO/IEC \n27001\nThird edition  \n2022-10\nReference number \nISO/IEC 27001:2022(E)\n© ISO/IEC 2022\n--``,,,,,``````,,,,,`,`,`,`,,`,-`-`,,`,,`,`,,`---"),
 Document(metadata={'producer': 'PDPreStamp v3.3', 'creator': 'Adobe InDesign 16.4 (Windows)', 'creati

In [49]:
texts = [doc.page_content for doc in chunks]
embeddings = embedding_manager.generate_embeddings(texts)

# Step 2: Add the documents + embeddings into ChromaDB
vectorstore.add_documents(chunks, embeddings)

# Optional sanity check
print("Vector store document count:", vectorstore.collection.count())

Generating embeddings for 76 texts...


Batches: 100%|██████████| 3/3 [00:06<00:00,  2.26s/it]


Generated embeddings with shape: (76, 384)
Adding 76 documents to vector store...
Successfully added 76 documents to vector store
Total documents in collection: 76
Vector store document count: 76


In [50]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [51]:
rag_retriever


In [ ]:
rag_retriever.retrieve("What are  Information security risk assessment ?")


Retrieving documents for query: 'What are  Information security risk assessment ?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_755d27f2_30',
  'content': 'ISO/IEC 27001:2022(E)\n6.1.2  Information security risk assessment\nThe organization shall define and apply an information security risk assessment process that:\na) establishes and maintains information security risk criteria that include:\n1) the risk acceptance criteria; and\n2) criteria for performing information security risk assessments;\nb) ensures that repeated information security risk assessments produce consistent, valid and \ncomparable results;\nc) identifies the information security risks:\n1) apply the information security risk assessment process to identify risks associated with \nthe loss of confidentiality, integrity and availability for information within the scope of the \ninformation security management system; and\n2) identify the risk owners;\nd) analyses the information security risks:\n1) assess the potential consequences that would result if the risks identified in 6.1.2 c) 1) were to \nmaterialize;',
  'metadata': {'au

Integration Vectordb Context pipeline With LLM output


In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
answer=rag_simple("What are  Information security risk assessment ??",rag_retriever,llm)
print(answer)